# Fantasy Footbal VBD Auction Creator and Optimizer
### DATA NOTE: SOurced via FantasyPros.com

1. Creates Value-based drafting values from 'Points above replacement' concept
2. Determines optimal split per cost on any position (eg. 80 to RB = 60 RB1; 20 RB2)
3. Runs optimal team splits (% on each position)

Remaining issues:
- QB values are wonky
    - Normalize for prior year values?
    - Manually adjust upwards?
- Optimizating of team splits has too many calculations
    - Narrow search ranges
    - Increase percentage increment?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv(r'C:\Users\stefa\OneDrive\Documents\Analysis Projects\Fantasy Modeling\Football\2021 Data\2021_projections.csv')

In [3]:
df = df.rename(columns={'position':'Pos', 'median_fpts':'Exp_PTS', 
                        'low_fpts':'Lo_PTS', 'high_fpts':'Hi_PTS', 
                        'name':'Player', 'team': 'Team' })
df = df.fillna(0)
df['Position_Rank'] = df.groupby('Pos')['Exp_PTS'].rank(ascending=False, method='first')
df['Pos'] = df['Pos'].str.upper()

In [4]:
df.head(3)

,Unnamed: 0,Player,Team,Pos,Exp_PTS,Hi_PTS,Lo_PTS,Position_Rank
0,0,Josh Allen,BUF,QB,379.6,438.8,371.2,1.0
1,1,Patrick Mahomes II,KC,QB,377.1,442.2,367.6,2.0
2,2,Kyler Murray,ARI,QB,357.3,414.4,352.0,3.0


### Define league settings

In [16]:
teams = 12
buyin = 200
qb_start, qb_bench = 2, 0.5
rb_start, rb_bench = 2.5, 1
wr_start, wr_bench = 2.5, 2
te_start, te_bench = 1, .25
bench_size = 5 ### actual 6, but calculated to optimize for 2 flex. 
d_start = 1
k_start = 1
rb_waiver = int(teams*(rb_start+rb_bench)+1)
wr_waiver = int(teams*(wr_start+wr_bench)+1)
#qb_waiver = int(teams*(qb_start+qb_bench)+1)
qb_waiver = 28
te_waiver = int(teams*(te_start+te_bench)+1)
op_waiver = int(teams*15+1)

## Run assumptions

In [17]:
starters = .96
### position multipliers based on prior years



# Replacement values

In [18]:
df[df['Pos']=='RB'].head(20)

,Unnamed: 0,Player,Team,Pos,Exp_PTS,Hi_PTS,Lo_PTS,Position_Rank
96,96,Christian McCaffrey,CAR,RB,301.4,338.7,247.4,1.0
97,97,Dalvin Cook,MIN,RB,279.5,307.0,250.6,2.0
98,98,Derrick Henry,TEN,RB,270.1,284.6,257.0,3.0
99,99,Alvin Kamara,NO,RB,238.4,279.2,217.6,4.0
100,100,Saquon Barkley,NYG,RB,235.3,253.8,217.8,5.0
101,101,Jonathan Taylor,IND,RB,229.5,254.2,214.0,6.0
102,102,Nick Chubb,CLE,RB,228.0,256.1,198.3,7.0
103,103,Aaron Jones,GB,RB,226.2,249.2,210.2,8.0
104,104,Ezekiel Elliott,DAL,RB,225.9,256.9,217.5,9.0
105,105,Joe Mixon,CIN,RB,214.9,239.8,192.7,10.0


In [19]:
# Find replacement-level RB.
#### Replacement = (rb_start+rb_bench)*teams+1
#### Look up points of replacement-level by pos. 
df_rb = df[df['Pos']=='RB']
repl_rb = df_rb.iloc[rb_waiver+1]['Exp_PTS']

df_qb = df[df['Pos']=='QB']
repl_qb = df_qb.iloc[qb_waiver+1]['Exp_PTS']

df_wr = df[df['Pos']=='WR']
repl_wr = df_wr.iloc[wr_waiver+1]['Exp_PTS']

df_te = df[df['Pos']=='TE']
repl_te = df_te.iloc[te_waiver+1]['Exp_PTS']



In [20]:
df_rb['PAR'] = np.max(df_rb['Exp_PTS']-0,0)
df_rb['PAR'] =df_rb.apply(lambda row: df_rb['Exp_PTS'] -repl_rb,axis=0)
df_rb.loc[df_rb['PAR']<0,'PAR'] = 0

df_wr['PAR'] = np.max(df_wr['Exp_PTS']-0,0)
df_wr['PAR'] =df_wr.apply(lambda row: df_wr['Exp_PTS'] -repl_wr,axis=0)
df_wr.loc[df_wr['PAR']<0,'PAR'] = 0

df_qb['PAR'] = np.max(df_qb['Exp_PTS']-0,0)
df_qb['PAR'] =df_qb.apply(lambda row: df_qb['Exp_PTS'] -repl_qb,axis=0)
df_qb.loc[df_qb['PAR']<0,'PAR'] = 0

df_te['PAR'] = np.max(df_te['Exp_PTS']-0,0)
df_te['PAR'] =df_te.apply(lambda row: df_te['Exp_PTS'] -repl_te,axis=0)
df_te.loc[df_te['PAR']<0,'PAR'] = 0


C:\Users\stefa\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\stefa\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\stefa\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [21]:
df_op2 = df[(df['Pos']!='DEF')&(df['Pos']!='K')]
repl_op = df_op2['Exp_PTS'].values[op_waiver]
df_op2['PAR'] = df_op2['Exp_PTS'] - repl_op
df_op2.loc[df_op2['PAR']<0,'PAR'] = 0
df_op2.head()

C:\Users\stefa\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Unnamed: 0,Player,Team,Pos,Exp_PTS,Hi_PTS,Lo_PTS,Position_Rank,PAR
0,0,Josh Allen,BUF,QB,379.6,438.8,371.2,1.0,342.6
1,1,Patrick Mahomes II,KC,QB,377.1,442.2,367.6,2.0,340.1
2,2,Kyler Murray,ARI,QB,357.3,414.4,352.0,3.0,320.3
3,3,Lamar Jackson,BAL,QB,349.4,406.8,327.9,4.0,312.4
4,4,Dak Prescott,DAL,QB,341.0,403.9,340.4,5.0,304.0


In [22]:
df_op = df_rb.append(df_qb, ignore_index=True)
df_op = df_op.append(df_te, ignore_index=True)
df_op = df_op.append(df_wr, ignore_index=True)
df_op.dropna()
total_par = np.sum(df_op['PAR'])
value_per_pt = teams * (buyin*starters) / total_par

In [23]:
print(value_per_pt)
teams*buyin*starters/df_op2['PAR'].sum()

0.24991051381341314


0.054141953434160194

In [24]:
df_op['Auction Value'] = np.max(df_op['PAR'],0)
df_op['Auction Value'] = df_op.apply(lambda row: df_op['PAR'] * value_per_pt,axis=0)
df_op['Downside'] = (df_op['Lo_PTS']-df_op['Exp_PTS'])*value_per_pt
df_op['Upside'] = (df_op['Hi_PTS']-df_op['Exp_PTS'])*value_per_pt
df_op.dropna()
df_op.sort_values(by=['Auction Value'], ascending=False)
df_pres = df_op[['Pos','Player','Team','Auction Value','Upside','Downside','Exp_PTS','Hi_PTS','Lo_PTS']]
df_pres = df_pres.sort_values(by=['Auction Value','Exp_PTS'], ascending=False)



In [25]:
#df_pres.to_csv(r'C:\Users\stefa\OneDrive\Documents\Analysis Projects\Fantasy Modeling\Football\2020 Data\Auction_Value_2020.csv')

In [26]:
df_pres = df_pres[df_pres['Exp_PTS']>0].dropna()
df_pres[df_pres['Pos']=='RB'][:30]

,Pos,Player,Team,Auction Value,Upside,Downside,Exp_PTS,Hi_PTS,Lo_PTS
0,RB,Christian McCaffrey,CAR,51.181673,9.321662,-13.495168,301.4,338.7,247.4
1,RB,Dalvin Cook,MIN,45.708633,6.872539,-7.222414,279.5,307.0,250.6
2,RB,Derrick Henry,TEN,43.359474,3.623702,-3.273828,270.1,284.6,257.0
3,RB,Alvin Kamara,NO,35.437311,10.196349,-5.198139,238.4,279.2,217.6
4,RB,Saquon Barkley,NYG,34.662588,4.623345,-4.373434,235.3,253.8,217.8
5,RB,Jonathan Taylor,IND,33.213107,6.172790,-3.873613,229.5,254.2,214.0
6,RB,Nick Chubb,CLE,32.838242,7.022485,-7.422342,228.0,256.1,198.3
7,RB,Aaron Jones,GB,32.388403,5.747942,-3.998568,226.2,249.2,210.2
8,RB,Ezekiel Elliott,DAL,32.313429,7.747226,-2.099248,225.9,256.9,217.5
9,RB,Joe Mixon,CIN,29.564414,6.222772,-5.548013,214.9,239.8,192.7


### Sanity Check on Auction Values
- Compare values for top positions vs. prior years.

In [ ]:
df_2020=

In [105]:
df_2019 = pd.read_csv(r'C:\Users\stefa\OneDrive\Documents\Analysis Projects\Fantasy Modeling\Football\2020 Data\Actual vs. Predicted 2019.csv')
df_2019.head()

,Pos,PosRank,Player,Team,Auction Value,Upside,Downside,Exp_PTS,Hi_PTS,Lo_PTS,Actual,Actual - Pred
0,QB,1,Patrick Mahomes,KC,41.776675,6.997519,-2.828784,339.6,363.1,330.1,41,-0.8
1,QB,2,Deshaun Watson,HOU,35.821340,5.985112,-3.126551,319.6,339.7,309.1,41,5.2
2,QB,3,Baker Mayfield,CLE,26.024814,6.908189,-2.888337,286.7,309.9,277.0,40,14.0
3,QB,4,Lamar Jackson,BAL,24.655087,9.945409,-7.027295,282.1,315.5,258.5,39,14.3
4,QB,5,Aaron Rodgers,GB,33.439206,8.099256,-3.870968,311.6,338.8,298.6,36,2.6


In [104]:
df_2019[df_2019['Pos']=='RB']

,Pos,PosRank,Player,Team,Auction Value,Upside,Downside,Exp_PTS,Hi_PTS,Lo_PTS,Actual,Actual - Pred
30,RB,1,David Johnson,ARI,32.337469,9.588089,-5.657568,214.4,246.6,195.4,56,23.7
31,RB,2,Saquon Barkley,NYG,49.995037,7.176179,-5.210918,273.7,297.8,256.2,55,5.0
32,RB,3,Ezekiel Elliott,DAL,46.183623,10.630273,-8.754342,260.9,296.6,231.5,54,7.8
33,RB,4,Christian McCaffrey,CAR,43.890819,9.766749,-5.836228,253.2,286.0,233.6,53,9.1
34,RB,5,Alvin Kamara,NO,42.729529,5.270471,-1.369727,249.3,267.0,244.7,53,10.3
...,...,...,...,...,...,...,...,...,...,...,...,...
86,RB,57,Damien Harris,NE,0.000000,9.588089,-5.717122,63.2,95.4,44.0,1,1.0
87,RB,58,Alexander Mattison,MIN,0.000000,12.595534,-6.193548,53.0,95.3,32.2,1,1.0
88,RB,59,Justice Hill,BAL,0.000000,12.625310,-5.062035,46.5,88.9,29.5,1,1.0
89,RB,60,Darwin Thompson,KC,0.000000,14.531017,-7.414392,39.2,88.0,14.3,1,1.0


In [106]:
tier_list = [3,6,12,24,140]
pos_list = ['QB', 'RB', 'WR', 'TE']

df_comp = pd.DataFrame()
for pos in pos_list:
    for tier in tier_list: 
        cur_proj = round(df_pres[df_pres['Pos']==pos]['Auction Value'].values[:tier].mean(),0)
        last_actual = round(df_2019[df_2019['Pos']==pos]['Actual'].values[:tier].mean(),0) 
        df_temp = pd.DataFrame(data=({'Pos':pos,
                                      'tier':tier, 
                                      'Proj':cur_proj, 
                                      'Last_act': last_actual}), index=[0])
        df_comp = df_comp.append(df_temp)
df_comp['ratio'] = df_comp['Last_act'] / df_comp['Proj'] 

In [107]:
pos_list = df_2019[df_2019['Pos']=='RB']['Auction Value'].values
pos_list = np.sort(pos_list)[::-1]
pos_list

array([49.99503722, 46.18362283, 43.89081886, 42.72952854, 32.33746898,
       31.83126551, 31.23573201, 30.66997519, 29.86600496, 29.18114144,
       27.81141439, 26.62034739, 26.44168734, 25.9057072 , 23.34491315,
       22.54094293, 21.26054591, 20.87344913, 20.33746898, 19.98014888,
       19.86104218, 19.05707196, 18.37220844, 15.03722084, 13.9057072 ,
       13.3101737 , 13.191067  ,  8.87344913,  7.02729529,  5.35980149,
        4.01985112,  3.27543424,  3.12655087,  2.67990074,  0.74441687,
        0.62531017,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ])

In [99]:
def adjuster(pos, pos_rank):
     return df_comp[(df_comp['Pos']==pos)&(df_comp['tier']>=pos_rank)]['ratio'].values[0]
    
def last_year_act(pos, pos_rank):
    try:
        act = df_2019[(df_2019['Pos']==pos)&(df_2019['PosRank']==pos_rank)]['Actual'].values[0]
    except:
        act = 0
    return act

def last_year_pred(pos, pos_rank):
    pos_list = df_2019[df_2019['Pos']==pos]['Auction Value'].values
    pos_list = np.sort(pos_list)[::-1]
    try:
        pred = pos_list[int(pos_rank)-1]
    except: 
        pred = 0 
    return pred

In [108]:
df_comp

,Pos,tier,Proj,Last_act,ratio
0,QB,3,47.0,41.0,0.872340
0,QB,6,43.0,39.0,0.906977
0,QB,12,38.0,36.0,0.947368
0,QB,24,30.0,28.0,0.933333
0,QB,140,10.0,22.0,2.200000
0,RB,3,41.0,55.0,1.341463
0,RB,6,35.0,53.0,1.514286
0,RB,12,30.0,44.0,1.466667
0,RB,24,24.0,33.0,1.375000
0,RB,140,4.0,15.0,3.750000


In [101]:
df_pres['pos_rank'] = df_pres.groupby('Pos')['Exp_PTS'].rank(axis=0, ascending=False, method='min')
df_pres['expected_auction'] = np.minimum(2,np.vectorize(adjuster)(df_pres['Pos'], df_pres['pos_rank']))*df_pres['Auction Value']
df_pres['last_year_posrank_act'] = np.vectorize(last_year_act)(df_pres['Pos'], df_pres['pos_rank'])
df_pres['last_year_posrank_pred'] = np.vectorize(last_year_pred)(df_pres['Pos'], df_pres['pos_rank'])
df_pres.head(10)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
df_pres = df_pres[['Pos','pos_rank','Player','Team','Exp_PTS','Auction Value','expected_auction','last_year_posrank_act','last_year_posrank_pred']]
df_pres

In [66]:
df_pres.to_csv(r'C:\Users\stefa\OneDrive\Documents\Analysis Projects\Fantasy Modeling\Football\2020 Data\Auction_Value_2020.csv')